1. Discuss the scenarios where multithreading is preferable to multiprocessing and scenarios wheremultiprocessing is a better choice.

> Multithreading, Multiprocessing

> Multithreading: Multiple threads within a single process that share memory. Best for I/O-bound tasks (e.g., waiting on disk or network operations), lightweight tasks, and real-time applications. In Python, multithreading is limited for CPU-bound tasks due to the Global Interpreter Lock (GIL).

> Multiprocessing: Multiple independent processes with separate memory. Best for CPU-bound tasks (e.g., computationally heavy tasks), as each process can run on a separate core, bypassing the GIL in Python. It’s also good for fault isolation, where process crashes don't affect others.

> When to Use Multithreading
I/O-bound tasks (e.g., file reading, network requests).
Tasks that require high responsiveness and low overhead.
Shared memory is needed between tasks (e.g., a simulation with shared state).

> When to Use Multiprocessing
CPU-bound tasks (e.g., image processing, data crunching).
Tasks requiring parallelism to fully utilize multiple CPU cores.
Tasks with large memory requirements or the need for fault isolation (e.g., separate processes for scraping websites).

2. Describe what a process pool is and how it helps in managing multiple processes efficiently.2. Describe what a process pool is and how it helps in managing multiple processes efficiently.

> What is a Process Pool?
A process pool is a collection of pre-created worker processes that are reused to handle tasks. Instead of creating new processes for each task, a pool assigns tasks to idle worker processes, improving efficiency.


> How It Works:
Pre-allocated processes: A fixed number of worker processes are created and kept ready.
Task assignment: Tasks are assigned to available processes in the pool.
Reuse: After completing a task, processes are returned to the pool for reuse.

> Benefits:
Reduced Overhead: Reusing processes avoids the cost of creating new ones.
Better Resource Utilization: Limits the number of processes, preventing system overload.
Concurrency and Parallelism: Efficiently manages multiple tasks, especially for CPU-bound work.

3. Explain what multiprocessing is and why it is used in Python programs.
> What is Multiprocessing?
Multiprocessing allows multiple processes to run simultaneously, each with its own memory space and resources. Unlike multithreading, where threads share memory, multiprocessing provides true parallelism, making it ideal for CPU-bound tasks.

> Why Use Multiprocessing in Python?
Bypasses the GIL: Python’s Global Interpreter Lock (GIL) limits multithreading for CPU-bound tasks. Multiprocessing avoids this by using separate processes, each with its own GIL.
Parallelism: Utilizes multiple CPU cores, speeding up computation-heavy tasks like data processing and scientific computations.

> When to Use Multiprocessing:
CPU-bound tasks like calculations or data processing.
Heavy computations that can be parallelized across multiple cores.

In [1]:
#4. Write a Python program using multithreading where one thread adds numbers to a list, and anotherthread removes numbers from the list. Implement a mechanism to avoid race conditions using  threading.Lock
import threading
import time

# Shared list
shared_list = []

# Lock to avoid race conditions
list_lock = threading.Lock()

# Producer thread: Adds numbers to the list
def producer():
    for i in range(5):
        time.sleep(1)  # Simulate work
        with list_lock:  # Lock the list before adding a number
            shared_list.append(i)
            print(f"Producer added {i}")

# Consumer thread: Removes numbers from the list
def consumer():
    while True:
        time.sleep(2)  # Simulate work
        with list_lock:  # Lock the list before accessing it
            if shared_list:
                num = shared_list.pop(0)
                print(f"Consumer removed {num}")
            else:
                print("Consumer found the list empty, waiting for items...")
                break  # Exit if the list is empty

# Create threads for producer and consumer
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

# Start the threads
producer_thread.start()
consumer_thread.start()

# Wait for threads to complete
producer_thread.join()
consumer_thread.join()

print("Program finished.")

Producer added 0
Producer added 1
Consumer removed 0
Producer added 2
Producer added 3
Consumer removed 1
Producer added 4
Consumer removed 2
Consumer removed 3
Consumer removed 4
Consumer found the list empty, waiting for items...
Program finished.


5. Describe the methods and tools available in Python for safely sharing data between threads and processes.

> Threading (Multithreading)
threading.Lock: A basic lock to ensure only one thread accesses shared data at a time.

> Example: with lock: shared_data += 1
threading.RLock: A reentrant lock, allowing a thread to acquire it multiple times.

> threading.Semaphore: Controls access to a resource by limiting the number of threads accessing it simultaneously.

queue.Queue: A thread-safe queue for safe data exchange between threads.

> Example: q.put(data), data = q.get()
threading.Event: Used for signaling between threads (e.g., one thread waits for an event to be set).

   > Multiprocessing (Processes)
multiprocessing.Queue: A safe FIFO queue for communication between processes.

> multiprocessing.Manager: Creates shared objects (e.g., lists, dictionaries) that can be safely accessed by multiple processes.

> multiprocessing.Value and multiprocessing.Array: Allows sharing simple values or arrays between processes.

> multiprocessing.Pipe: A two-way communication channel between two processes.

> multiprocessing.Lock: Ensures mutual exclusion, preventing multiple processes from modifying shared data at once.

6. Discuss why it’s crucial to handle exceptions in concurrent programs and the techniques available for doing so.
  > Why Handling Exceptions in Concurrent Programs is Crucial
> Prevents Crashes: Unhandled exceptions can terminate threads or processes unexpectedly, leading to program crashes.
> Prevents Cascading Failures: Proper exception handling ensures one thread/process failure doesn’t affect others.
> Resource Management: Ensures resources (e.g., files, memory) are managed properly, even when errors occur.
> Helps Debugging: Capturing and logging exceptions makes it easier to diagnose issues in concurrent code.

In [2]:
# 1. Try-Except Blocks:
import threading

def worker():
    try:
        raise ValueError("Error")
    except Exception as e:
        print(f"Error: {e}")

thread = threading.Thread(target=worker)
thread.start()
thread.join()

Error: Error


In [3]:
# 2. Global Exception Handlers (sys.excepthook):
import sys
import threading

def handle_exception(exc_type, exc_value, exc_traceback):
    print(f"Unhandled exception: {exc_value}")

sys.excepthook = handle_exception

def worker():
    raise ValueError("Error")

thread = threading.Thread(target=worker)
thread.start()
thread.join()

Exception in thread Thread-13 (worker):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-fbf2a60fd5a9>", line 11, in worker
ValueError: Error


In [4]:
# 3. concurrent.futures:
from concurrent.futures import ThreadPoolExecutor

def worker(x):
    if x == 2:
        raise ValueError("Error")
    return x * 2

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(worker, i) for i in range(4)]
    for future in futures:
        try:
            print(future.result())
        except Exception as e:
            print(f"Exception: {e}")

0
2
Exception: Error
6


In [5]:
# 4. Multiprocessing:
from multiprocessing import Process, Queue

def worker(q):
    try:
        raise ValueError("Error")
    except Exception as e:
        q.put(f"Error: {e}")

if __name__ == "__main__":
    q = Queue()
    p = Process(target=worker, args=(q,))
    p.start()
    p.join()
    print(q.get())

Error: Error


In [ ]:
# 5. Logging:
import logging
import threading

logging.basicConfig(level=logging.ERROR)

def worker():
    try:
        raise ValueError("Error")
    except Exception as e:
        logging.error(f"Error: {e}")

thread = threading.Thread(target=worker)
thread.start()
thread.join()

In [7]:
# 7. Create a program that uses a thread pool to calculate the factorial of numbers from 1 to 10 concurrently. Use concurrent.futures.ThreadPoolExecutor to manage the threads.
import concurrent.futures
import math

# Function to calculate factorial
def calculate_factorial(n):
    return math.factorial(n)

# Main function to manage threads using ThreadPoolExecutor
def main():
    # Create a ThreadPoolExecutor with 5 worker threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Submit tasks to the executor for numbers 1 to 10
        numbers = range(1, 11)
        futures = [executor.submit(calculate_factorial, num) for num in numbers]

        # Wait for all the futures to complete and print the results
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            print(result)

if __name__ == "__main__":
    main()

720
2
362880
1
3628800
6
120
5040
40320
24


In [8]:
# 8. Create a Python program that uses multiprocessing.Pool to compute the square of numbers from 1 to 10 in parallel. Measure the time taken to perform this computation using a pool of different sizes (e.g., 2, 4, 8 processes).
import multiprocessing
import time

# Function to compute the square of a number
def square(n):
    return n * n

# Function to measure the time taken using different pool sizes
def compute_squares(pool_size):
    # Start measuring time
    start_time = time.time()

    # Create a pool of workers
    with multiprocessing.Pool(pool_size) as pool:
        # Compute squares in parallel
        results = pool.map(square, range(1, 11))

    # Calculate and print the time taken
    end_time = time.time()
    print(f"Time taken with {pool_size} processes: {end_time - start_time:.4f} seconds")

    # Return the results for display
    return results

def main():
    # Run the computation with different pool sizes and display the results
    for pool_size in [2, 4, 8]:
        print(f"\nComputing squares with {pool_size} processes...")
        results = compute_squares(pool_size)
        print("Results:", results)

if __name__ == "__main__":
    main()


Computing squares with 2 processes...
Time taken with 2 processes: 0.1169 seconds
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

Computing squares with 4 processes...
Time taken with 4 processes: 0.1941 seconds
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

Computing squares with 8 processes...
Time taken with 8 processes: 0.1982 seconds
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
